In [2]:
# imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import pyplot
%matplotlib inline

In [3]:
#Daten einlesen
df_orig = pd.read_parquet("D:/Eclipse Python/DataScience/res/event_ano_new.parquet.gzip")

In [4]:
df = df_orig.set_index("event_timestamp")
df.sort_index(inplace=True)

In [4]:
line_mapping = {}
for i in range(0,26):
    line_mapping.update({"line_{}".format(i):i})

df["an_line"] = df.an_line.apply(lambda x: line_mapping[x] if x in line_mapping.keys() else x)

In [5]:
cell_mapping = {}
for i in range(0,13):
    cell_mapping.update({"cell_{}".format(i):i})

df["an_cell"] = df.an_cell.apply(lambda x: cell_mapping[x] if x in cell_mapping.keys() else x)

In [6]:
robot_mapping = {}
for i in range(0,188):
    robot_mapping.update({"robot_{}".format(i):i})

df["an_robot"] = df.an_robot.apply(lambda x: robot_mapping[x] if x in robot_mapping.keys() else x)

In [7]:
message_severity_mapping = {"Error":1,"Warning":2,"Information":3}

df["message_severity"] = df.message_severity.apply(lambda x: message_severity_mapping[x] if x in message_severity_mapping.keys() else x)

In [8]:
def dfExtractMethod(df,extract_value):
    df_extract = df.loc[df['an_line'] == extract_value]
    return df_extract

In [9]:
df_extract = dfExtractMethod(df,1)

In [10]:
def dfExtractErrorWarningInformationMethod(df,extract_value):
    df_extract = df.loc[df['message_severity'] == extract_value]
    return df_extract

In [11]:
df_extract_index = df_extract.reset_index()

In [12]:
def dfExtractErrorWarningInformationAnd10RowsUpAndDownMethod(df,extract_value):
    data = []
    for i in df:
        data.append(i)

    number = 0
    for h in df.values:
        number = number + 1
        if(h[3] == extract_value):
            for j in range(0,10):
                data.append(df.values[number-j])
                    
            data.append(h)
            for j in range(0,10):
                data.append(df.values[number+j])
     
    return(data)

In [5]:
ErrorsPerLinesPattern = dfExtractErrorWarningInformationAnd10RowsUpAndDownMethod(df_extract_index.head(100),1)
myErrorsPerLinesPatternArray = np.asarray(ErrorsPerLinesPattern)
df_ErrorsPerLinesPattern = pd.DataFrame(myErrorsPerLinesPatternArray)
df_ErrorsPerLinesPattern

NameError: name 'dfExtractErrorWarningInformationAnd10RowsUpAndDownMethod' is not defined